In [0]:

bronze_container = "abfss://bronze@andrewstockmarket.dfs.core.windows.net/"  # Bronze data stored in Parquet or Delta
silver_container = "abfss://silver@andrewstockmarket.dfs.core.windows.net/"  # Target location for Silver Delta table
gold_container = "abfss://gold@andrewstockmarket.dfs.core.windows.net/"  # Target location for Silver Delta table


# Your Delta folder path
silver_path =   silver_container + "delta-tables/main"
dimSymbol_path = gold_container + "delta-tables/dim-symbol"
dimDate_path = gold_container + "delta-tables/dim-date"
fact_path =  gold_container + "delta-tables/fact-daily-summary"




# ## re-create table 
# truncate_schema = spark.read.format("delta").load(silver_path).schema
# truncate_silver = spark.createDataFrame([], schema= truncate_schema)
# truncate_silver.write.format("delta").mode("overwrite").save(silver_path)

In [0]:
%sql
 
-- use  schema silver 
-- drop  schema   if  exists hive_metastore.silver cascade 

In [0]:
%sql
show  CATALOGs;


In [0]:
spark.sql( """
CREATE EXTERNAL LOCATION gold
URL 'abfss://gold@andrewstockmarket.dfs.core.windows.net/'
WITH (STORAGE CREDENTIAL dev_credentials)
""")


In [0]:
# catalog = "hive_metastore" 
# schema_list = ["silver", "gold"]  

# for schema in schema_list:
        
#        tables = spark.sql(f"show tables in {catalog}.{schema}").collect()

#        for table in tables :
#            if not table.isTemporary:  
#             print(table.isTemporary)
#             display(spark.sql(f"DESCRIBE EXTENDED  {catalog}.{schema}.{table.tableName}"))
     


### Silver

In [0]:
%sql
drop table  stocks.silver.silver_table

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE stocks.silver.silver_table (
    Symbol                   STRING,
    ExchangeName             STRING,
    Currency                 STRING,
    Type                     STRING,
    ExchangeTimeZone         STRING,
    Volume                   INTEGER,
    High                     DECIMAL,
    Low                      DECIMAL,
    Close                    DECIMAL,
    Open                     DECIMAL,
    Date                     DATE,
    __CurrentFlag            BOOLEAN,
    __DeletedFlag            BOOLEAN,
    __EffectiveStartDateTime TIMESTAMP,
    __EffectiveEndDateTime   TIMESTAMP,
    __lastmodified           TIMESTAMP,
    __HashKey                STRING,
    __HashValue              STRING,
    __CreatedBatchLogId      STRING,
    __UpdatedBatchLogId      STRING,
    __CreateDateTime         TIMESTAMP,
    __UpdateDateTime         TIMESTAMP
)
USING DELTA
LOCATION '{silver_path}'
""")



In [0]:
# # 3) Verify registration and schema
# display( spark.sql("SHOW TABLES"))
# display( spark.sql("DESCRIBE silver.silver_table"))

### Dim Date

In [0]:

# 2) Create the table with full schema, pointing at that folder
spark.sql(f"""
CREATE TABLE stocks.gold.dim_date (
    
    DateID              INTEGER,
    Date                DATE,
    Day                 INTEGER,
    DayOfWeek           VARCHAR(20),
    DayOfWeekNumber     INTEGER,
    MonthName           VARCHAR(20),
    MonthNumber         INTEGER,
    Year                INTEGER,
    YearMonth           INTEGER
   
)
USING DELTA
LOCATION '{dimDate_path}'
""")

# 3) Verify registration and schema
spark.sql("SHOW TABLES").show(truncate=False)
spark.sql("DESCRIBE delta_table").show(truncate=False)


### Dim Symbol

In [0]:

# 2) Create the table with full schema, pointing at that folder
spark.sql(f"""
CREATE TABLE stocks.gold.DimSymbol (
    SymbolSID                BIGINT GENERATED ALWAYS AS IDENTITY(START WITH 1 increment by 1) ,
    Symbol                   STRING,
    ExchangeName             STRING,
    Currency                 STRING,
    Type                     STRING,
    ExchangeTimeZone         STRING,
    __CurrentFlag            BOOLEAN,
    __DeletedFlag            BOOLEAN,
    __EffectiveStartDateTime TIMESTAMP,
    __EffectiveEndDateTime   TIMESTAMP,
    __BusinessKeyHash        VARCHAR(64),
    __Hash1Type              VARCHAR(64),
    __Hash2Type              VARCHAR(64),
    __CreatedBatchLogId      VARCHAR(50),
    __UpdatedBatchLogId      VARCHAR(50),
    __CreateDateTime         TIMESTAMP,
    __UpdateDateTime         TIMESTAMP
)
USING DELTA
LOCATION '{dimSymbol_path}'
""")

# # 3) Verify registration and schema
# spark.sql("SHOW TABLES").show(truncate=False)
# spark.sql("DESCRIBE dimsymbol_table").show(truncate=False)


### Fact Daily Summary

In [0]:

# 2) Create the table with full schema, pointing at that folder
spark.sql(f"""
CREATE TABLE stocks.gold.FactDaily (
    
    TransactionSID           BIGINT GENERATED ALWAYS AS IDENTITY(START WITH 1 increment by 1),
    SymbolSID                BIGINT,
    Volume                   INTEGER,
    High                     DECIMAL,
    Low                      DECIMAL,
    Close                    DECIMAL,
    Open                     DECIMAL,
    DateID                   INTEGER,
    __DeletedFlag            BOOLEAN,
    __FactKeyHash            VARCHAR(64),
    __CreatedBatchLogId      VARCHAR(50),
    __CreateDateTime         TIMESTAMP
)
USING DELTA
LOCATION '{fact_path}'
""")

# 3) Verify registration and schema
# spark.sql("SHOW TABLES").show(truncate=False)
# spark.sql("DESCRIBE delta_table").show(truncate=False)
